In [ ]:
# QR vectorized form 찾기
# QR costfunction 구현

# QR gradient vectorized form 찾기
# QR coordinate vectorized form 구현

# lasso QR costfunction 구현

### Linear Programming_QR

In [1]:
# library import
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.optimize import linprog  # Linear Programming 

In [42]:
car_df = pd.read_csv("Cars93.csv")

In [43]:
car_df = car_df.iloc[:, 1:]

In [44]:
# price와 origin의 관계 볼 예정
# 필요 컬럼 추출 및 origin 더미화
car_df = car_df[['Price', 'Origin']]
car_df = pd.get_dummies(car_df, columns=['Origin'], drop_first=True)

In [46]:
y = car_df['Price'].values
y.shape

(93,)

In [47]:
X = car_df['Origin_non-USA']
X = sm.add_constant(X)
X = X.values
X.shape

C:\Users\sypar\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


(93, 2)

In [48]:
n = X.shape[0]  # the number of data
p = X.shape[1]  # the number of parameters

In [50]:
qs = [0.1, 0.25, 0.5, 0.75, 0.9] #quantiles

### QR loss :  argmin(β)∑ρ_θ(y_i - x_i*β)
### ρ_θ = θ[y]+ + (1-θ)[-y]+
### [y]+ = max{0, y}
### y = (y1, y2, .., yn) 일 때, s1 = [y-Xβ]+, s2 = [Xβ-y]+
### y-Xβ = s1 - s2
### β = [β]+ - [-β]+

### argmin(β){θ*1^t*s1 + (1-θ)*1^t*s2 | y = Xβ+s1-s2}
### A = [X, -X, I, -I]
### ψ = ([β]^t+, [-β]^t+, s1^t, s2^t)^t
### c = (0p^t, 0p^t, θ*1n^t, (1-θ)*1^t )^t

### minimize(β){c^tψ }
### subject to Aψ  = y, ψ >=0

In [52]:
## ψ 구하기
for q in qs:
    A_eq = np.hstack([X, -X, np.identity(n), -np.identity(n)]) #A
    b_eq = y 
    c = np.array([0]*(2*p) + [q]*n + [1-q]*n) #c
    res = linprog(c, A_eq=A_eq, b_eq=b_eq,method='highs')
    
    x_plus = res.x[:p]
    x_minus = res.x[p:2*p]
    x = x_plus-x_minus
    print(f'q : {q}, intercept : {x[0]}, slope : {round(x[1],3)}')

q : 0.1, intercept : 11.1, slope : -2.5
q : 0.25, intercept : 13.5, slope : -1.9
q : 0.5, intercept : 16.3, slope : 2.8
q : 0.75, intercept : 20.7, slope : 6.0
q : 0.9, intercept : 34.3, slope : -0.4


### Subgradient

In [ ]:
def subgradient(alpha, max_its, initial_params, tolerance):
    if initial_params is None:
        X_tX = X.T.dot(X)
        X_tX_inv = np.linalg.inv(X_tx)
        w = X_tx_inv.dot(X.T.dot(X))
    else:
        w = initial_params
        
    gradient = grad(cost)
    its = 0
    ws = [w]
    cost_values = [cost(w)]
    while its < max_its:
        its += 1
        
        grad_eval = gradient(w)
        w = w - alpha*grad_eval
        cost_val = cost(w)
        ws.append(w)
        cost_values.append(cost_val)
        
    min_cost = min(cost_values)
    min_idx = cost_values.index(min_cost)
    
    params = ws[min_idx]
    cost = min_cost
    

In [14]:
# Libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits import mplot3d
from matplotlib import animation
from IPython.display import HTML, Image

%matplotlib inline
plt.style.use('seaborn-white')

In [15]:
x = np.linspace(0, 1, 40) # 0~1 값들 40개 생성

In [17]:
noise = 1*np.random.uniform(size = 40) # noise 40개 생성

In [19]:
y = np.sin(x*1.5 *np.pi) # 삼각함수 값

In [22]:
y_noise = (y+noise).reshape(-1,1)  # 1열로 변경

In [24]:
y_noise = y_noise - y_noise.mean()

In [46]:
X = np.vstack((2*x, x**2)).T   # 2x, x^2 // vstack : 배열 세로 결합, T : transposed

In [33]:
initial_theta = np.array([4, -4.]).reshape(-1,1)

In [ ]:
def costfunction(X, y, theta):
    #initialsation of useful values
    m = np.size(y)
    
    #cost function in vectorized form
    h = X @ theta       # (40, 2) @ (2,1) => (40, 1)
    J = float((1./(2*m)) * (h - y).T @ (h - y))
    return J

In [63]:
X.shape

(40, 2)

In [ ]:
def coordinate_descent(theta, X, y, alpha = 0.03, num_iters = 20):
    # coordinate gradient descetn for linear regression 
    #initialisation of useful values
    m,n = X.shape    #(40, 2)
    J_history = []
    theta_0_hist, theta_1_hist = [], []    #for plotting afterwards
    
    for i in range(num_iters):
        for j in range(n):   # n = 2
            #coordinate descent in vectorized form
            h = X @ theta
            gradient = (x[:, j] @ (h - y))
            theta[j] = theta[j] - alpha * gradient
            
            # saving values for plots
            J_history.append(costfunction(X, y, theta))
            theta_0_hist.append(theta[0,0])
            theta_1_hist.append(theta[1,0])
            
    return theta, J_history, theta_0_hist, theta_1_hist


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
x = np.linspace(10, 100, 1000)
y = np.linspace(10, 100, 1000)

In [13]:
def func(x, y, param):
    return param[0] * x + param[1] * y

In [ ]:
def cosft(x, y, param):
    return np.sum((X.dot(param) - y)**2)/(2* len(y))

In [ ]:
z = func(x, y, [5, 8]) + np.random.normal(0., 10.)
z = z.reshape(-1, 1)

In [ ]:
interc = np.ones(1000)
X = np.concatenate([interc.reshape(-1, 1), x.reshape(-1, 1), y.reshape(-1, 1)], axis=1)

In [ ]:
#param = np.random.randn(3).T
param = np.array([2, 2, 2])

In [ ]:
def gradient_descent(X, y, param, eta=0.01, iter=100):
    cost_history = [0] * iter

    for iteration in range(iter):
        h = X.dot(param)
        loss = h - y
        gradient = X.T.dot(loss)/(2 * len(y))
        param = param - eta * gradient
        cost = costf(X, y, param)
        #print(cost)
        cost_history[iteration] = cost

    return param, cost_history

In [ ]:
def coordinate_descent(X, y, param, iter=100):
    cost_history = [0] * iter

    for iteration in range(iter):
        for i in range(len(param)):
            dele = np.dot(np.delete(X, i, axis=1), np.delete(param, i, axis=0))
            param[i] = np.dot(X[:,i].T, (y - dele))/np.sum(np.square(X[:,i]))
            cost = costf(X, y, param)
            #print(cost)
            cost_history[iteration] = cost

    return param, cost_history


ret, xret = gradient_descent(X, z, param)
cret, cxret = coordinate_descent(X, z, param)

plt.plot(range(len(xret)), xret, label="GD")
plt.plot(range(len(cxret)), cxret, label="CD")
plt.legend()
plt.show()